<a href="https://colab.research.google.com/github/YashNigam65/gitfolder/blob/master/measuring_perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# ------------------------------
# Choose a causal language model
# ------------------------------
MODEL_NAME = "gpt2"  # You can change this to any causal LM on HF hub

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.eval()  # evaluation mode

if torch.cuda.is_available():
    model.to("cuda")


def sentence_perplexity(text: str) -> float:
    """Compute perplexity of a single sentence/string."""
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # For perplexity, we pass labels = input_ids so the model
    # tries to predict each next token given previous ones.
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss  # this is average cross-entropy per token

    ppl = math.exp(loss.item())
    return ppl


def corpus_perplexity(texts):
    """Compute perplexity over a list of texts (properly weighted by length)."""
    total_log_likelihood = 0.0
    total_tokens = 0

    for text in texts:
        inputs = tokenizer(text, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = {k: v.to("cuda") for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            # loss is mean over tokens in this example
            n_tokens = inputs["input_ids"].numel()
            total_log_likelihood += -outputs.loss.item() * n_tokens
            total_tokens += n_tokens

    # Average negative log-likelihood per token
    avg_neg_log_likelihood = total_log_likelihood / total_tokens
    ppl = math.exp(-avg_neg_log_likelihood)
    return ppl


if __name__ == "__main__":
    text = "The quick brown fox jumps over the lazy dog."
    print("Sentence:", text)
    print("Perplexity:", sentence_perplexity(text))

    corpus = [
        "The quick brown fox jumps over the lazy dog.",
        "Language models estimate the probability of a sequence of words.",
        "Perplexity is a common metric to evaluate them."
    ]
    print("Corpus perplexity:", corpus_perplexity(corpus))